![Img](https://app.theheadstarter.com/static/hs-logo-opengraph.png)

# Headstarter AI Agent Workshop

#### **Skills: OpenAI, Groq, Llama, OpenRouter**

## **To Get Started:**
1. [Get your Groq API Key](https://console.groq.com/keys)
2. [Get your OpenRouter API Key](https://openrouter.ai/settings/keys)
3. [Get your OpenAI API Key](https://platform.openai.com/api-keys)


### **Interesting Reads**
- [Sam Altman's Blog Post: The Intelligence Age](https://ia.samaltman.com/)
- [What LLMs cannot do](https://ehudreiter.com/2023/12/11/what-llms-cannot-do/)
- [Chain of Thought Prompting](https://www.promptingguide.ai/techniques/cot)
- [Why ChatGPT can't count the number of r's in the word strawberry](https://prompt.16x.engineer/blog/why-chatgpt-cant-count-rs-in-strawberry)


## During the Workshop
- [Any code shared during the workshop will be posted here](https://docs.google.com/document/d/1hPBJt_4Ihkj6v667fWxVjzwCMS4uBPdYlBLd2IqkxJ0/edit?usp=sharing)

# Install necessary libraries

In [1]:
! pip install openai groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.0/383.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.3 MB/s eta 0:00:00


# Set up Groq, OpenRouter, & OpenAI clients

In [32]:
from openai import OpenAI
from google.colab import userdata
import os
import json
from groq import Groq
from typing import List, Dict, Any, Callable
import ast
import io
import sys

# Retrieve API keys and set environment variables
groq_api_key = userdata.get("GROQ_API_KEY")
os.environ['GROQ_API_KEY'] = groq_api_key

openrouter_api_key = userdata.get("OPENROUTER_API_KEY")
os.environ['OPENROUTER_API_KEY'] = openrouter_api_key

# Initialize clients for Groq and OpenRouter
groq_client = Groq(api_key=os.getenv('GROQ_API_KEY'))

openrouter_client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.getenv("OPENROUTER_API_KEY")
)

### Define functions to easily query and compare responses from OpenAI, Groq, and OpenRouter

In [33]:
# Function to get LLM responses from Groq or OpenRouter
def get_llm_response(client, prompt, json_mode=False):
    if client == "groq":
        models = [
            "llama-3.1-8b-instant",
            "llama-3.1-70b-versatile",
            "llama3-70b-8192",
            "llama3-8b-8192",
            "gemma2-9b-it"
        ]

        for model in models:
            try:
                kwargs = {
                    "model": model,
                    "messages": [{"role": "user", "content": prompt}]
                }
                if json_mode:
                    kwargs["response_format"] = {"type": "json_object"}

                response = groq_client.chat.completions.create(**kwargs)
                return response.choices[0].message.content

            except Exception as e:
                print(f"Error with Groq model {model}: {e}")
                continue

        print("All Groq models failed to provide a response.")
        return None

    elif client == "openrouter":
        models = ["meta-llama/llama-3.1-8b-instruct:free"]

        for model in models:
            try:
                kwargs = {
                    "model": model,
                    "messages": [{"role": "user", "content": prompt}]
                }
                if json_mode:
                    kwargs["response_format"] = {"type": "json_object"}

                response = openrouter_client.chat.completions.create(**kwargs)
                return response.choices[0].message.content

            except Exception as e:
                print(f"Error with OpenRouter model {model}: {e}")
                continue

        print("OpenRouter failed to provide a response.")
        return None

    else:
        raise ValueError(f"Invalid client: {client}")

# Function to evaluate and compare responses from Groq and OpenRouter
def evaluate_responses(prompt, reasoning_prompt=False):
    if reasoning_prompt:
        prompt = f"{prompt}\n\n{reasoning_prompt}."

    groq_response = get_llm_response("groq", prompt)
    openrouter_response = get_llm_response("openrouter", prompt)

    print(f"Groq Response: {groq_response}")
    print(f"\n\nOpenRouter Response: {openrouter_response}")

In [34]:
prompt1 = "How many r's are in the word 'strawberry'?"
evaluate_responses(prompt1)

Groq Response: There are 2 r's in the word 'strawberry'.


OpenRouter Response: There are 2 r's in the word 'strawberry'.


In [36]:
prompt2 = "Whats bigger 9.9 or 9.11'?"
evaluate_responses(prompt2)


Groq Response: Both numbers are very close. However, 9.11 is a decimal greater than 9.9.


OpenRouter Response: The answer is... 9.11 is bigger.

To see why, let's compare the two numbers:

9.9 and 9.11 are both decimal numbers. When we compare them, we can see that 9.11 is slightly bigger than 9.9 because the second number has a hundredth (0.11) that is greater than the second number of the first number (0.9).

In other words, 9.11 is bigger by 0.02, or two tenths of a decimal place.


# Agent Architecture

[![](https://mermaid.ink/img/pako:eNqFUslugzAQ_ZWRz8kPILUVCZClVdQmqdTK5ODCFFDAjry0iSD_XmOTJodK5WS_Zd7M4JZkIkcSkEKyQwnbKOVgvzf6qlDCi0F52sF4fA-hJ0IaGi24aIRREBbItacn9LlmnKPced2kR7s1aiO5AmU-NFN71cG0fRLiALqUwhQlbAbi7F1TVyuia2RKXItFDo5pmOnqBo5dhgcDmKFlmEaY2oE6SOgvwHgO8REzM5B_2n1kxYsOZtSrxSUocfkzf5m5y5zGX6w27Cqau3IDOpTU8gR3kLBa2Y4W7QqP-jLywzDywtnesd_NLbISHSxpUnFWQ8jVt_0b8VFLll0Tl66TR-q3DLfaf3vaSoMukYxIg7JhVW4fQdvbUqJLbDAlgT3mTO5TkvKz1TG7ks2JZyTQ1j0i5pDb9UYVs2-nIcFnP-YFjfPKNjqA5x9CM8YW?type=png)](https://mermaid.live/edit#pako:eNqFUslugzAQ_ZWRz8kPILUVCZClVdQmqdTK5ODCFFDAjry0iSD_XmOTJodK5WS_Zd7M4JZkIkcSkEKyQwnbKOVgvzf6qlDCi0F52sF4fA-hJ0IaGi24aIRREBbItacn9LlmnKPced2kR7s1aiO5AmU-NFN71cG0fRLiALqUwhQlbAbi7F1TVyuia2RKXItFDo5pmOnqBo5dhgcDmKFlmEaY2oE6SOgvwHgO8REzM5B_2n1kxYsOZtSrxSUocfkzf5m5y5zGX6w27Cqau3IDOpTU8gR3kLBa2Y4W7QqP-jLywzDywtnesd_NLbISHSxpUnFWQ8jVt_0b8VFLll0Tl66TR-q3DLfaf3vaSoMukYxIg7JhVW4fQdvbUqJLbDAlgT3mTO5TkvKz1TG7ks2JZyTQ1j0i5pDb9UYVs2-nIcFnP-YFjfPKNjqA5x9CM8YW)


![agent_architecture_v2](https://github.com/user-attachments/assets/a65b6db9-bef1-4579-aed3-01444ce40544)

### To create our AI Agent, we will define the following functions:

1. **Planner:** This function takes a user's query and breaks it down into smaller, manageable subtasks. It returns these subtasks as a list, where each one is either a reasoning task or a code generation task.

2. **Reasoner:** This function provides reasoning on how to complete a specific subtask, considering both the overall query and the results of any previous subtasks. It returns a short explanation on how to proceed with the current subtask.

3. **Actioner:** Based on the reasoning provided for a subtask, this function decides whether the next step requires generating code or more reasoning. It then returns the chosen action and any necessary details to perform it.

4. **Evaluator:** This function checks if the result of the current subtask is reasonable and aligns with the overall goal. It returns an evaluation of the result and indicates whether the subtask needs to be retried.

5. **generate_and_execute_code:** This function generates and executes Python code based on a given prompt and memory of previous steps. It returns both the generated code and its execution result.

6. **executor:** Depending on the action decided by the "actioner," this function either generates and executes code or returns reasoning. It handles the execution of tasks based on the action type.

7. **final_answer_extractor:** After all subtasks are completed, this function gathers the results from previous steps to extract and provide the final answer to the user's query.

8. **autonomous_agent:** This is the main function that coordinates the process of answering the user's query. It manages the entire sequence of planning, reasoning, action, evaluation, and final answer extraction to produce a complete response.

In [39]:
def planner(user_query) -> List[str]:
  prompt = f"""Given the user's query: '{user_query}', break down the query into as few subtasks as possibile in orde to answer the question.

  Each subtask should be either a reasonsing task or a code generation task. Never duplicate a task.

  Here are the only 2 actions taht can be taken for each subtask:
    - generater_code: This actions involves generating Python code and execting it inorder to make a calcualtion or a vaerfication
    - reasosnign: This asction invovles proving reasosnign for waht to do to complete the subtask

  Each subtask should begin with either "reasosning" or "generate_code".

  Keep in mind the overall goal of answeing the useers query thoughout the planning process

  Return the result as a JSON list of strings, where each string is a subtask.

  Here is an example JSON repsosne:
  {{
      "subtasks": ["Subtask 1", "Subtask 2", "Subtask 3"]
  }}
  """

  response = json.loads(get_llm_response("openrouter", prompt, json_mode=True))
  return response["subtasks"]

In [40]:
query = "How many r's are in the word 'strawberry'?"
subtasks = planner(query)
subtasks

["Reasoning: Identify the word in the user's query to count the number of r's",
 "Reasoning: Determine the correct count of r's in the word 'strawberry'",
 "Generate_code: Write a Python function to count the number of 'r's in the word 'strawberry'"]

In [41]:
def reasoner(user_query: str, subtasks: List[str], current_subtask: str, memory: List[Dict[str, Any]]) -> str:
   prompt = f"""Given the user's query (long-term goal): '{user_query}'

   Here are all the subtasks to complete in order to answer the user's query:
   <subtasks>
       {json.dumps(subtasks)}
   </subtasks>

   Here is the short-term memory (result of previous subtasks):
   <memory>
       {json.dumps(memory)}
   </memory>

   The current subtask to complete is:
   <current_subtask>
       {current_subtask}
   </current_subtask>

   - Provide concise reasoning on how to execute the current subtask, considering previous results.
   - Prioritize explicit details over assumed patterns
   - Avoid unnecessary complications in problem-solving

   Return the result as a JSON object with 'reasoning' as a key.

   Example JSON response:
   {{
       "reasoning": "2 sentences max on how to complete the current subtask."
   }}
   """

   response = json.loads(get_llm_response("groq", prompt, json_mode=True))
   return response["reasoning"]

In [42]:
reasoner_ouput = reasoner(query, subtasks, subtasks[0], [])


"Parse the user's query for the specific word, in this case 'strawberry', considering it as a string. Use a text extraction technique, such as tokenization or regular expressions, to isolate the word of interest and make it the target for counting 'r's."

In [26]:


def actioner(user_query: str, subtasks: List[str], current_subtask: str, reasoning: str, memory: List[Dict[str, Any]]) -> Dict[str, Any]:
   prompt = f"""Given the user's query (long-term goal): '{user_query}'

   The subtasks are:
   <subtasks>
       {json.dumps(subtasks)}
   </subtasks>

   The current subtask is:
   <current_subtask>
       {current_subtask}
   </current_subtask>

   The reasoning for this subtask is:
   <reasoning>
       {reasoning}
   </reasoning>

   Here is the short-term memory (result of previous subtasks):
   <memory>
       {json.dumps(memory)}
   </memory>

   Determine the most appropriate action to take:
       - If the task requires a calculation or verification through code, use the 'generate_code' action.
       - If the task requires reasoning without code or calculations, use the 'reasoning' action.

   Consider the overall goal and previous results when determining the action.

   Return the result as a JSON object with 'action' and 'parameters' keys.  The 'parameters' key should always be a dictionary with 'prompt' as a key.

   Example JSON responses:

   {{
       "action": "generate_code",
       "parameters": {{"prompt": "Write a function to calculate the area of a circle."}}
   }}

   {{
       "action": "reasoning",
       "parameters": {{"prompt": "Explain how to complete the subtask."}}
   }}
   """

   response = json.loads(get_llm_response("groq", prompt, json_mode=True))
   return response


In [44]:
actioner(query, subtasks, subtasks[1], reasoner_ouput, []) # Added an empty list as the memory argument.


{'action': 'reasoning',
 'parameters': {'prompt': "The word 'strawberry' contains two 'r's in a sequence, one more single 'r'. Now we need to count the individual 'r's which are 3."}}

In [27]:
def evaluator(user_query: str, subtasks: List[str], current_subtask: str, action_info: Dict[str, Any], execution_result: Dict[str, Any], memory: List[Dict[str, Any]]) -> Dict[str, Any]:
   prompt = f"""Given the user's query (long-term goal): '{user_query}'

   The subtasks to complete to answer the user's query are:
   <subtasks>
       {json.dumps(subtasks)}
   </subtasks>

   The current subtask to complete is:
   <current_subtask>
       {current_subtask}
   </current_subtask>

   The result of the current subtask is:
   <result>
       {action_info}
   </result>

   The execution result of the current subtask is:
   <execution_result>
       {execution_result}
   </execution_result>

   Here is the short-term memory (result of previous subtasks):
   <memory>
       {json.dumps(memory)}
   </memory>


   Evaluate if the result is a reasonable answer for the current subtask, and makes sense in the context of the overall query.

   Return a JSON object with 'evaluation' (string) and 'retry' (boolean) keys.

   Example JSON response:
   {{
       "evaluation": "The result is a reasonable answer for the current subtask.",
       "retry": false
   }}
   """


   response = json.loads(get_llm_response("groq", prompt, json_mode=True))
   return response


In [28]:
def final_answer_extractor(user_query: str, subtasks: List[str], memory: List[Dict[str, Any]]) -> str:
   prompt = f"""Given the user's query (long-term goal): '{user_query}'

   The subtasks completed to answer the user's query are:
   <subtasks>
       {json.dumps(subtasks)}
   </subtasks>

   The memory of the thought process (short-term memory) is:
   <memory>
       {json.dumps(memory)}
   </memory>

   Extract the final answer that directly addresses the user's query, from the memory.
   Provide only the essential information without unnecessary explanations.

   Return a JSON object with 'finalAnswer' as a key.

   Here is an example JSON response:
   {{
       "finalAnswer": "The final answer to the user's query, addressing all aspects of the question, based on the memory provided",
   }}
   """

   response = json.loads(get_llm_response("groq", prompt, json_mode=True))
   return response["finalAnswer"]




In [48]:
def generate_and_execute_code(prompt: str, user_query: str, memory: List[Dict[str, Any]]) -> Dict[str, Any]:
   code_generation_prompt = f"""

   Generate Python code to implement the following task: '{prompt}'

   Here is the overall goal of answering the user's query: '{user_query}'

   Keep in mind the results of the previous subtasks, and use them to complete the current subtask.
   <memory>
       {json.dumps(memory)}
   </memory>



   Here are the guidelines for generating the code:
       - Return only the Python code, without any explanations or markdown formatting.
       - The code should always print or return a value
       - Don't include any backticks or code blocks in your response. Do not include in your response, just give me the code.
       - Do not ever use the input() function in your code, use defined values instead.
       - Do not ever use NLP techniques in your code, such as importing nltk, spacy, or any other NLP library.
       - Don't ever define a function in your code, just generate the code to execute the subtask.
       - Don't ever provide the execution result in your response, just give me the code.
       - If your code needs to import any libraries, do it within the code itself.
       - The code should be self-contained and ready to execute on its own.
       - Prioritize explicit details over assumed patterns
       - Avoid unnecessary complications in problem-solving
   """

   generated_code = get_llm_response("groq", code_generation_prompt)


   print(f"\n\nGenerated Code: start|{generated_code}|END\n\n")

   old_stdout = sys.stdout
   sys.stdout = buffer = io.StringIO()

   # Add import statement for the 're' module before executing the generated code
   exec('import re\n' + generated_code)

   sys.stdout = old_stdout
   output = buffer.getvalue()

   print(f"\n\n***** Execution Result: |start|{output.strip()}|end| *****\n\n")

   return {
       "generated_code": generated_code,
       "execution_result": output.strip()
   }

   def executor(action: str, parameters: Dict[str, Any], user_query: str, memory: List[Dict[str, Any]]) -> Any:
    if action == "generate_code":
        print(f"Generating code for: {parameters['prompt']}")
        return generate_and_execute_code(parameters["prompt"], user_query, memory)
    elif action == "reasoning":
        return parameters["prompt"]
    else:
        return f"Action '{action}' not implemented"

In [49]:


def autonomous_agent(user_query: str) -> List[Dict[str, Any]]:
   memory = []
   subtasks = planner(user_query)

   print("User Query:", user_query)
   print(f"Subtasks: {subtasks}")

   for subtask in subtasks:
       max_retries = 1
       for attempt in range(max_retries):

           reasoning = reasoner(user_query, subtasks, subtask, memory)
           action_info = actioner(user_query, subtasks, subtask, reasoning, memory)


           print(f"\n\n ****** Action Info: {action_info} ****** \n\n")

           execution_result = executor(action_info["action"], action_info["parameters"], user_query, memory)

           print(f"\n\n ****** Execution Result: {execution_result} ****** \n\n")
           evaluation = evaluator(user_query, subtasks, subtask, action_info, execution_result, memory)

           step = {
               "subtask": subtask,
               "reasoning": reasoning,
               "action": action_info,
               "evaluation": evaluation
           }
           memory.append(step)

           print(f"\n\nSTEP: {step}\n\n")

           if not evaluation["retry"]:
               break

           if attempt == max_retries - 1:
               print(f"Max retries reached for subtask: {subtask}")

   final_answer = final_answer_extractor(user_query, subtasks, memory)
   return final_answer


In [51]:
result = autonomous_agent(query)
print("FiNAL ANSWER: ", result)

# OpenAI o1-preview model getting trivial questions wrong

- [Link to Reddit post](https://www.reddit.com/r/ChatGPT/comments/1ff9w7y/new_o1_still_fails_miserably_at_trivial_questions/)
- Links to ChatGPT threads: [(1)](https://chatgpt.com/share/66f21757-db2c-8012-8b0a-11224aed0c29), [(2)](https://chatgpt.com/share/66e3c1e5-ae00-8007-8820-fee9eb61eae5)
- [Improving reasoning in LLMs through thoughtful prompting](https://www.reddit.com/r/singularity/comments/1fdhs2m/did_i_just_fix_the_data_overfitting_problem_in/?share_id=6DsDLJUu1qEx_bsqFDC8a&utm_content=2&utm_medium=ios_app&utm_name=ioscss&utm_source=share&utm_term=1)
